## Parse and extract text from `warc` file
1. With `fastwarc` and `resiliparse`
    1. https://resiliparse.chatnoir.eu/en/stable/index.html

In [ ]:
from fastwarc import ArchiveIterator
from resiliparse.extract.html2text import extract_plain_text
from resiliparse.parse.encoding import detect_encoding

def extract_text(record):
    byte_string = record.reader.read()
    encoding = detect_encoding(byte_string)
    html_content = byte_string.decode(encoding=encoding)
    extracted_text = extract_plain_text(html_content)
    return extracted_text

warc_file = "/home/azureuser/localfiles/cs336-assignment4-data-mine/cs336_data/CC-MAIN-20250417135010-20250417165010-00065.warc.gz"
iterator = ArchiveIterator(open(warc_file, "rb"), func_filter=lambda r: r.headers.get('WARC-Identified-Payload-Type') == 'text/html')

# record = next(iterator)
# print(extract_text(record))

In [ ]:
# To get random records for ensuing tests
# Method: Skip randomly through iterator
import random

def get_random_records(iterator, N=20, skip_prob=0.9):
    """Skip records randomly and return extracted text immediately"""
    random_data = []
    for i, record in enumerate(iterator):
        if len(random_data) >= N:
            break
        if random.random() > skip_prob:
            # Read and process immediately while record is still fresh
            extracted_text = extract_text(record)
            random_data.append((i, extracted_text))
    return random_data

## Language identification

In [ ]:
import fasttext
model = fasttext.load_model("lid.176.bin")

# sanity check
model.predict("Hello world.")

In [ ]:
def language_generator():
    for i, text in random_data:
        print(f"=== Record {i} ===")
        print(text[:200] + "..." if len(text) > 200 else text)
        lang, score = model.predict(text.replace("\n", " "))
        print(f"Language: {lang[0]}, Score: {score[0]:.4f}")
        print("-" * 50)
        yield 

# Get random records with extracted text
random_data = get_random_records(iterator)

# Create the generator
lang_gen = language_generator()


In [ ]:
# Run this cell multiple times to get one result at a time
next(lang_gen)

## PII
### email address
- Length: The total length of an email address is capped at 320 characters, with 64 for the username and 255 for the domain.
- Spaces: Spaces are not allowed.
- Case sensitivity: Email addresses are generally not case-sensitive, meaning User@Example.com is the same as user@example.com.

- Special characters:
    - Periods (.), hyphens (-), and underscores (_) are often allowed in the local part.
    - They cannot be the first or last character of the local part and cannot appear consecutively (e.g., john..doe@example.com is invalid).
    - In the domain, hyphens are allowed but not at the beginning or end of a label (a part between periods). 

### US phone number
1. use pattern
    - (\+1\s*)? - optional `+1` followed by optional spaces
    - \(? - optional opening parenthesis
    - \d{3} - 3 digits
    - \)? - optional closing parenthesis
    - [\s-]? - at most one space or hyphen (the ? means zero or one)
    - \s* - zero or more additional spaces
    - \d{3} - 3 digits
    - [\s-]? - at most one space or hyphen
    - \s* - zero or more additional spaces
    - \d{4} - 4 digits

### IP address
- Use `\b` to avoid matching things like:
    - 1.2.3.4.5 (too many octets)
    - version1.2.3.4 (prefix attached)
    - 1.2.3.4th (suffix attached)

In [ ]:
from mask_pii import mask_email, mask_phone, mask_ip

record = next(iterator)
text = extract_text(record)
print(text)

In [ ]:
text, mask_email_counts = mask_email(text)
text, mask_phone_counts = mask_phone(text)
text, mask_ip_counts = mask_ip(text)

if any([counts > 0 for counts in [mask_email_counts, mask_phone_counts, mask_ip_counts]]):
    print(text)

## Harmful content

In [ ]:
import fasttext

model = fasttext.load_model("/home/azureuser/localfiles/cs336-assignment4-data-mine/cs336_data/jigsaw_fasttext_bigrams_nsfw_final.bin")


In [ ]:
text = (
    "SUCK MY C*CK WIKIPEDIA EDITORS...F*CKING *SSH*LE DORKS. "
    "JUST TRYING TO MAKE THE SITE BETTER YOU UPTIGHT C*NTS"
)
# text = "Umm, theres no actual article for prostitution ring.  - Crunch Captain."
text = "操你妈 你个傻逼 干死你"

model.predict(text)

In [ ]:
print(text)

In [ ]:
from harmful_content import classify_nsfw, classify_toxic_speech

found = False
while found == False:
    record = next(iterator)
    try:
        text = extract_text(record)
    except UnicodeDecodeError:
        continue
    # print(text)
    is_nsfw, score_nsfw = classify_nsfw(text)
    is_toxic, score_toxic = classify_toxic_speech(text)

    if is_nsfw == "nsfw" or is_toxic == "toxic":
        print(text)
        print("="*80)
        print("Harmful content detected!")
        print(is_nsfw, is_toxic)
        print(score_nsfw, score_toxic)

        found = True


## Quality Rule
- Contain less than 50 or more than 100,000 words.
- Have a mean word length outside the range of 3 to 10 characters.
- Have more than 30% of lines ending with an ellipsis (“...”).
- Contain less than 80% of words with at least one alphabetic character.

In [ ]:
import nltk
import re

def valid_words(text):
    # num of words requirement
    words = nltk.word_tokenize(text)
    if len(words) < 50 or len(words) > 1e5:
        return False

    # word length requirement
    word_lens = [len(word) for word in words]
    mean_len = sum(word_lens)/len(words)
    if mean_len < 3.0 or mean_len > 10.0:
        return False

    # at least one alphabetic character
    n_valid_words = sum(1 for word in words if re.search(r'[a-zA-Z]', word))
    if n_valid_words / len(words) < 0.8:
        return False

    return True

def valid_lines(text):
    lines = re.split(r"\n+", text)    
    n_valid_lines = sum(1 for line in lines if not line.endswith("..."))
    if n_valid_lines / len(lines) > 0.7:
        return True
    return False

def gopher_quality_filter(text):
    return valid_words(text) and valid_lines(text)


text = "This should definitely be a valid input text and of high quality according to Gopher rules.\n" *10
gopher_quality_filter(text)
# True

## Quality Classifier

- To get positive texts; the filtering might be a bit cheating...
    - `cat enwiki-20240420-extracted_urls.txt | grep "https://en.wikipedia.org/wiki" > enwiki-20240420-extracted_urls_subset.txt`
    - `wget --tries=2 --timeout=5 -i enwiki-20240420-extracted_urls_subset.txt --warc-file=subsampled_positive_urls -O /dev/null`

In [ ]:
from fastwarc import ArchiveIterator
from fastwarc.warc import WarcRecord
from resiliparse.extract.html2text import extract_plain_text
from resiliparse.parse.encoding import detect_encoding

def extract_text(record):
    byte_string = record.reader.read()
    encoding = detect_encoding(byte_string)
    try:
        html_content = byte_string.decode(encoding=encoding)
    except UnicodeDecodeError:
        return ""
    extracted_text = extract_plain_text(html_content)
    return extracted_text


In [ ]:
def is_html_record(record: WarcRecord) -> bool:
    try:
        return record.http_headers.get('Content-Type', '').startswith('text/html')
    except:
        return False

warc_file = "/home/azureuser/localfiles/cs336-assignment4-data-mine/cs336_data/subsampled_positive_urls.warc.gz"
iterator = ArchiveIterator(open(warc_file, "rb"))

from language_identification import identify_language
positive_texts = []
for record in iterator:
    if is_html_record(record):
        text = extract_text(record)
        if len(text) > 512:  # very low bar for wikipedia articles
            lang, score = identify_language(text)
            if lang == "en" and score > 0.8:
                positive_texts.append(text)

print(f"Raw number of positive texts: {len(positive_texts)}")

In [ ]:
import random
from language_identification import identify_language

def get_random_english_records_as_negative_texts(iterator, N=500, skip_prob=0.8):
    """Skip records randomly and return extracted text immediately"""
    random_data = []
    for record in iterator:
        if len(random_data) >= N:
            break
        if random.random() > skip_prob:
            # Read and process immediately while record is still fresh
            extracted_text = extract_text(record)
            lang, score = identify_language(extracted_text)
            if lang == "en" and score > 0.9:
                random_data.append(extracted_text)
    return random_data

warc_file = "/home/azureuser/localfiles/cs336-assignment4-data-mine/cs336_data/CC-MAIN-20250417135010-20250417165010-00065.warc.gz"
iterator = ArchiveIterator(open(warc_file, "rb"), func_filter=lambda r: r.headers.get('WARC-Identified-Payload-Type') == 'text/html')
negative_texts = get_random_english_records_as_negative_texts(iterator)

print(f"Raw number of negative texts: {len(negative_texts)}")

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def chunk_text_simple(text, tokenizer, max_length=512, stride=64):
    """Simpler version using return_overflowing_tokens"""
    tokens = tokenizer(
        text,
        max_length=max_length,
        stride=stride,
        truncation=True,
        return_overflowing_tokens=True,
        return_tensors="pt",
        padding="max_length"
    )
    
    return [
        {
            'input_ids': tokens['input_ids'][i].tolist(),
            'attention_mask': tokens['attention_mask'][i].tolist()
        }
        for i in range(len(tokens['input_ids']))
    ]

In [ ]:
positive_chunks = [chunk_text_simple(text, tokenizer) for text in positive_texts]
positive_chunks = [chunk for chunks in positive_chunks for chunk in chunks]
print(len(positive_chunks))

negative_chunks = [chunk_text_simple(text, tokenizer) for text in negative_texts]
negative_chunks = [chunk for chunks in negative_chunks for chunk in chunks]
print(len(negative_chunks))

In [ ]:
# Shuffle chunks before splitting
import random

random.seed(42)  # for reproducibility
random.shuffle(negative_chunks)
random.shuffle(positive_chunks)

# Create train/valid/test splits
n_neg = len(negative_chunks)
n_pos = len(positive_chunks)

print(f"Total negative chunks: {n_neg}")
print(f"Total positive chunks: {n_pos}")

# Split indices
train_neg = negative_chunks[:1000]
valid_neg = negative_chunks[1000:1200]

train_pos = positive_chunks[:1000]
valid_pos = positive_chunks[1000:1200]

# Create datasets
from datasets import Dataset

train_chunks = train_neg + train_pos
train_labels = [0] * len(train_neg) + [1] * len(train_pos)

valid_chunks = valid_neg + valid_pos
valid_labels = [0] * len(valid_neg) + [1] * len(valid_pos)

ds_train = Dataset.from_dict({
    "input_ids": [chunk['input_ids'] for chunk in train_chunks],
    "attention_mask": [chunk['attention_mask'] for chunk in train_chunks],
    "label": train_labels
})

ds_valid = Dataset.from_dict({
    "input_ids": [chunk['input_ids'] for chunk in valid_chunks],
    "attention_mask": [chunk['attention_mask'] for chunk in valid_chunks],
    "label": valid_labels
})

print(f"\nTrain: {len(ds_train)} samples (neg: {sum(1 for l in train_labels if l == 0)}, pos: {sum(1 for l in train_labels if l == 1)})")
print(f"Valid: {len(ds_valid)} samples (neg: {sum(1 for l in valid_labels if l == 0)}, pos: {sum(1 for l in valid_labels if l == 1)})")

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
training_args = TrainingArguments(
    output_dir="quality_classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_valid,
    processing_class=tokenizer,
)

trainer.train()

In [ ]:
# prediction
with open("/home/azureuser/localfiles/cs336-assignment4-data-mine/tests/fixtures/high_quality_wiki_reference.txt") as f:
    hq_text = f.read()
with open("/home/azureuser/localfiles/cs336-assignment4-data-mine/tests/fixtures/low_quality_cc.txt") as f:
    lq_text = f.read()


In [ ]:
hq_chunks = chunk_text_simple(hq_text, tokenizer)
lq_chunks = chunk_text_simple(lq_text, tokenizer)
# positive_chunks = [chunk for chunks in positive_chunks for chunk in chunks]
# print(len(positive_chunks))

In [ ]:
# Run inference on lq_text
import torch

model.eval()
predictions = []  # Fresh list each time

with torch.no_grad():
    for chunk in hq_chunks:
        # Convert to tensors and add batch dimension
        inputs = {
            'input_ids': torch.tensor([chunk['input_ids']]).cuda(),
            'attention_mask': torch.tensor([chunk['attention_mask']]).cuda()
        }
        
        # Get model predictions
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        
        predictions.append({
            'label': torch.argmax(probs, dim=-1).item(),
            'prob_negative': probs[0][0].item(),
            'prob_positive': probs[0][1].item()
        })

# Count predictions
positive_count = sum(1 for p in predictions if p['label'] == 1)
negative_count = sum(1 for p in predictions if p['label'] == 0)
total_chunks = len(predictions)

positive_ratio = positive_count / total_chunks
negative_ratio = negative_count / total_chunks

print(f"Total chunks: {total_chunks}")
print(f"Positive chunks: {positive_count} ({positive_ratio:.2%})")
print(f"Negative chunks: {negative_count} ({negative_ratio:.2%})")
print(f"\nFinal prediction: {'POSITIVE (high quality)' if positive_ratio > 0.5 else 'NEGATIVE (low quality)'}")

print(f"\nPer-chunk predictions:")
for i, pred in enumerate(predictions):
    print(f"Chunk {i+1}: {id2label[pred['label']]} (pos: {pred['prob_positive']:.4f})")

## Deduplication
### Exact line dedpulication

In [24]:
import os
dir = "/home/azureuser/localfiles/cs336-assignment4-data-mine/tests/fixtures/documents_with_line_duplicates"
dir_tgt = "/home/azureuser/localfiles/cs336-assignment4-data-mine/cs336_data"
files = [os.path.join(dir, f) for f in os.listdir(dir)]


In [21]:
import re
import hashlib

line_dict = {}
for file in files:
    with open(file, 'r') as f:
        content = f.read()
    
    lines = re.split(r'\n+', content)

    for i, line in enumerate(lines):
        if not line.strip():  # skip empty lines
            continue
        hash = hashlib.md5(line.encode()).hexdigest()
        file_basename = os.path.basename(file)
        # if hash not in line_dict:
        #     line_dict[hash] = [file_name]
        # else:
        #     if file_name not in line_dict[hash]:
        #         line_dict[hash].append(file_name)
        if hash not in line_dict:
            line_dict[hash] = {
                "line_id": [i],
                "file_basenames": [file_basename]
            }
        else:
            line_dict[hash]["line_id"].append(i)
            line_dict[hash]["file_basenames"].append(file_basename)

print(line_dict)
# line_dict = {k:list(set(v)) for k, v in line_dict.items()}

{'fd4c1f199cde9a998612f5de0a292e29': {'line_id': [0], 'file_basenames': ['doc5.txt']}, 'a48331fd9a20853375a5f9008858091f': {'line_id': [0, 0], 'file_basenames': ['doc4.txt', 'doc3.txt']}, '6548b245bba14aff59a41ea60098e88d': {'line_id': [1, 1], 'file_basenames': ['doc4.txt', 'doc3.txt']}, 'f9ff8a2ee66a24cc2a715a2d345a7a1c': {'line_id': [2], 'file_basenames': ['doc4.txt']}, '9c6c6c00ff2c041741c41fcee56d6846': {'line_id': [0, 0], 'file_basenames': ['doc2.txt', 'doc1.txt']}, 'a4ac8aeedce3b7e9f2ac1b07527aaa56': {'line_id': [2], 'file_basenames': ['doc3.txt']}, 'b9013db2f2b976d2a223fda39a8fab4b': {'line_id': [3], 'file_basenames': ['doc3.txt']}}


In [22]:
file_dict = {}
for k, v in line_dict.items():
    line_ids, file_basenames = v.values()
    if len(set(file_basenames)) == 1:
        continue
    for line_id, file_basename in zip(line_ids, file_basenames):
        if file_basename not in file_dict:
            file_dict[file_basename] = [line_id]
        else:
            file_dict[file_basename].append(line_id)

file_dict

{'doc4.txt': [0, 1], 'doc3.txt': [0, 1], 'doc2.txt': [0], 'doc1.txt': [0]}

In [25]:
import shutil
for file in files:
    file_basename = os.path.basename(file)
    file_tgt = os.path.join(dir_tgt, file_basename)
    if file_basename not in file_dict:
        shutil.copy(file, file_tgt)
        continue

    with open(file, 'r') as f:
        content = f.read()
    
    lines = re.split(r'\n+', content)
    lines2rm_ids = file_dict[file_basename]

    new_lines = [lines[i] for i in range(len(lines)) if i not in lines2rm_ids]
    with open(file_tgt, 'w') as f:
        f.write("\n".join(new_lines))
